Download Dataset

In [1]:
from IPython.display import clear_output

In [2]:
!kaggle datasets download -d josephnelson/chess-piece-images-and-bounding-boxes
!unzip chess-piece-images-and-bounding-boxes.zip
clear_output()

Covert Annotations xml to txt

In [3]:
from xml.dom import minidom
import os
import glob

In [4]:
classes = ['white-rook', 'white-knight', 'white-bishop', 'white-king', 'white-queen', 
           'white-pawn', 'black-rook', 'black-knight', 'black-bishop', 'black-king', 
           'black-queen', 'black-pawn']
dictionary = {j:i for i,j in enumerate(classes)} 
print(dictionary)

{'white-rook': 0, 'white-knight': 1, 'white-bishop': 2, 'white-king': 3, 'white-queen': 4, 'white-pawn': 5, 'black-rook': 6, 'black-knight': 7, 'black-bishop': 8, 'black-king': 9, 'black-queen': 10, 'black-pawn': 11}


In [5]:
def convert_coordinates(size, box):
    dw = 1.0/size[0]
    dh = 1.0/size[1]
    x = (box[0]+box[1])/2.0
    y = (box[2]+box[3])/2.0
    w = box[1]-box[0]
    h = box[3]-box[2]
    x = x*dw    
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [6]:
def convert_xml2yolo(dictionary,path):
    for fname in glob.glob(f"{path}/*.xml"):
        print(fname)
        xmldoc = minidom.parse(fname)
        fname_out = (fname[:-4]+'.txt')
        with open(fname_out, "w") as f:
            itemlist = xmldoc.getElementsByTagName('object')
            size = xmldoc.getElementsByTagName('size')[0]
            width = int((size.getElementsByTagName('width')[0]).firstChild.data)
            height = int((size.getElementsByTagName('height')[0]).firstChild.data)
            for item in itemlist:
                classid =  (item.getElementsByTagName('name')[0]).firstChild.data
                if classid in dictionary:
                    label_str = str(dictionary[classid])
                else:
                    label_str = "-1"
                    print ("warning: label '%s' not in look-up table" % classid)

                # get bbox coordinates
                xmin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmin')[0]).firstChild.data
                ymin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymin')[0]).firstChild.data
                xmax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmax')[0]).firstChild.data
                ymax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymax')[0]).firstChild.data
                b = (float(xmin), float(xmax), float(ymin), float(ymax))
                bb = convert_coordinates((width,height), b)

                f.write(label_str + " " + " ".join([("%.6f" % a) for a in bb]) + '\n')

        print ("wrote %s" % fname_out)

In [7]:
path = "/content/export"
convert_xml2yolo(dictionary,path)
clear_output()

In [8]:
!mkdir images
!mkdir yolo_annotations

In [9]:
!cp /content/export/*.txt yolo_annotations/
!cp /content/export/*.jpg images/

In [10]:
print(len(os.listdir("yolo_annotations/")))

289


In [11]:
print(len(os.listdir("/content/images")))

289


Structure into yolo format

In [12]:
!mkdir data
!mkdir data/images data/labels
!mkdir data/images/train data/images/val
!mkdir data/labels/train data/labels/val

In [13]:
import os
all_ = [i for i in os.listdir('/content/images/')]
train = all_[:250]
test = all_[250:]
len(all_),len(train),len(test)

(289, 250, 39)

In [14]:
!cp /content/images/*jpg data/images/train/
!cp /content/images/*jpg data/images/val/

In [15]:
!cp yolo_annotations/* data/labels/train/
!cp yolo_annotations/* data/labels/val/

In [16]:
for i in os.listdir("/content/data/images/train"):
  if i not in train:
    os.remove(f"/content/data/images/train/{i}")
for i in os.listdir("/content/data/images/val"):
  if i not in test:
    os.remove(f"/content/data/images/val/{i}")

In [ ]:
len(os.listdir("/content/data/images/train")),len(os.listdir("/content/data/images/val"))

In [17]:
for i in os.listdir("/content/images"):
  if i not in train:
    os.remove(f"/content/data/labels/train/{i.split('.')[0]}.txt")
for i in os.listdir("/content/images"):
  if i not in test:
    os.remove(f"/content/data/labels/val/{i.split('.')[0]}.txt")

In [18]:
len(os.listdir("/content/data/labels/train")),len(os.listdir("/content/data/labels/val"))

(250, 39)

Yolo V5

In [19]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!pip install -r requirements.txt
clear_output()

In [20]:
import torch
from IPython.display import Image, clear_output  # to display images
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)


Create a data.yaml

train: ../data/images/train

val: ../data/images/val

nc: 12

names: ['white-rook', 'white-knight', 'white-bishop', 'white-king', 'white-queen', 
           'white-pawn', 'black-rook', 'black-knight', 'black-bishop', 'black-king', 
           'black-queen', 'black-pawn']

In [21]:
!python train.py --img 640 --batch 4 --epochs 30 --data /content/data.yaml \
 --weights yolov5s.pt --nosave --cache 

YOLOv5 v4.0-10-g1d1c056 torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079.75MB)

Namespace(adam=False, batch_size=4, bucket='', cache_images=True, cfg='', data='/content/data.yaml', device='', epochs=30, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=True, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', single_cls=False, sync_bn=False, total_batch_size=4, weights='yolov5s.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2021-01-12 11:52:48.820811: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0

Test Images

In [22]:
!python detect.py --source /content/data/images/val/  --weights runs/train/exp/weights/last.pt --conf 0.4

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='/content/data/images/val/', update=False, view_img=False, weights=['runs/train/exp/weights/last.pt'])
YOLOv5 v4.0-10-g1d1c056 torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079.75MB)

Fusing layers... 
Model Summary: 224 layers, 7083577 parameters, 0 gradients, 16.4 GFLOPS
image 1/39 /content/data/images/val/055b79dd8db4c43e1a23be6095aaf624.jpg: 448x640 Done. (0.011s)
image 2/39 /content/data/images/val/05ad7223827a29a8283f6c4b2490f52f.jpg: 448x640 1 white-kings, 4 white-pawns, 2 black-kings, 5 black-pawns, Done. (0.012s)
image 3/39 /content/data/images/val/0c09b79cff39932c59ecc745dd827906.jpg: 448x640 1 white-kings, 2 white-pawns, 1 black-kings, 1 black-pawns, Done. (0.010s)
image 4/39 /content/data/images/val/1877a28e4c5f5c1ea68aca66f4e85d95.jpg: 448x640 1 white-kings, 8 white-pawns, 1 bl

Export

In [ ]:
!pip install onnx
!pip install coremltools
clear_output()

In [ ]:
!python models/export.py --weights runs/train/exp/weights/last.pt --img 640 --batch 1
clear_output()